In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from rocketpy import Environment, SolidMotor, Rocket, Flight
from datetime import datetime, timedelta
from rocketpy.plots.compare import CompareFlights
import datetime
import time
import ipywidgets as widgets
from IPython.display import display


q1 = widgets.ToggleButton(
        value=False,
        description='Custom Launch Location?',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Using Custom Location',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
output = widgets.Output()
display(q1, output)
with output:
    print('Defalt Location')


latitu = widgets.FloatText(
    value=32.990254,
    description='latitude:',
    disabled=False
)
longtitu = widgets.FloatText(
    value=-106.974998,
    description='longitude',
    disabled=False
)
elevat = widgets.FloatText(
    value=1400,
    description='levation',
    disabled=False
)

inclina = widgets.FloatText(
    value=85,
    description='Inclination',
    disabled=False
)
Headin = widgets.FloatText(
    value=0,
    description='Heading',
    disabled=False
)


def on_value_change(change):
    if q1.value:
        output.clear_output()
        with output:
            print('Custom Location')

            display(latitu)

            display(longtitu)
            
            display(elevat)

    else:
        output.clear_output()
        with output:
            print('Defalt Location')



q1.observe(on_value_change, names='value')

con = widgets.Button(description="Confirm")
display(con)

locat = widgets.Button(description='Set Location')
    
def on_button_clicked(b):
    output.clear_output()
    q1.close()
    con.close()
    with output:
        print('latitude: ', latitu.value, '  longitude: ', longtitu.value, '  elevation: ', elevat.value)
        display(locat)

def locat_clicked(b):
    output.clear_output()
    locat.close()
    with output:
        global env
        env = Environment(latitude=float(latitu.value), longitude=float(longtitu.value), elevation=float(elevat.value))
        print('Location Selected')
        display(inclina)
        display(Headin)
        display(rocket_define)

con.on_click(on_button_clicked)
locat.on_click(locat_clicked)

rocket_define = widgets.Button(description='Rocket Define')


def rocket_clicked(b):
    global calisto
    
    Pro75M1670 = SolidMotor(
        thrust_source="data/motors/Cesaroni_M1670.eng",
        dry_mass=1.815,
        dry_inertia=(0.125, 0.125, 0.002),
        nozzle_radius=33 / 1000,
        grain_number=5,
        grain_density=1815,
        grain_outer_radius=33 / 1000,
        grain_initial_inner_radius=15 / 1000,
        grain_initial_height=120 / 1000,
        grain_separation=5 / 1000,
        grains_center_of_mass_position=0.397,
        center_of_dry_mass_position=0.317,
        nozzle_position=0,
        burn_time=3.9,
        throat_radius=11 / 1000,
        coordinate_system_orientation="nozzle_to_combustion_chamber",
    )

    calisto = Rocket(
        radius=127 / 2000,
        mass=14.426,
        inertia=(6.321, 6.321, 0.034),
        power_off_drag="data/calisto/powerOffDragCurve.csv",
        power_on_drag="data/calisto/powerOnDragCurve.csv",
        center_of_mass_without_motor=0,
        coordinate_system_orientation="tail_to_nose",
    )

    rail_buttons = calisto.set_rail_buttons(
        upper_button_position=0.0818,
        lower_button_position=-0.618,
        angular_position=45,
    )

    calisto.add_motor(Pro75M1670, position=-1.255)

    nose_cone = calisto.add_nose(
        length=0.55829, kind="vonKarman", position=1.278
    )

    fin_set = calisto.add_trapezoidal_fins(
        n=4,
        root_chord=0.120,
        tip_chord=0.060,
        span=0.110,
        position=-1.04956,
        cant_angle=0.5,
        airfoil=("data/calisto/NACA0012-radians.csv","radians"),
    )

    tail = calisto.add_tail(
        top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
    )

    Main = calisto.add_parachute(
        "Main",
        cd_s=10.0,
        trigger=800,
        sampling_rate=105,
        lag=1.5,
        noise=(0, 8.3, 0.5),
    )

    Drogue = calisto.add_parachute(
        "Drogue",
        cd_s=1.0,
        trigger="apogee",
        sampling_rate=105,
        lag=1.5,
        noise=(0, 8.3, 0.5),
    )

    output.clear_output()
    
    global inclina
    global Headin
    
    inclina = inclina.value
    Headin = Headin.value
    
    with output:
        print('Rocket Defined')
        print('Inclination: ', inclina, '  Heading: ', Headin)
        display(f1)
        display(f2)
        display(f3)

rocket_define.on_click(rocket_clicked)



f1 = widgets.Button(description='1.Launch Date Recommand')
f2 = widgets.Button(description='2.Inclination/Heading comparison')
f3 = widgets.Button(description='3.Rail Length Comparison')

def f1_clicked(b):
    output.clear_output()
    with output:
        print('1.Launch Date Recommand')
        print()
        flights = []
        err = 0
        for i in range(1,8,1):
            if err == 5:
                break
            A12 = True
            err = 0
            while A12:
                try:
                    print("Calculating Day{0} Weather...".format(i))
                    tomorrow = datetime.date.today() + datetime.timedelta(days=i)
                    env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
                    env.set_atmospheric_model(type="Forecast", file="GFS")
                    print("Launching Day{0}".format(i))
                    flight = Flight(
                        environment=env,
                        rocket=calisto,
                        rail_length=5.2,
                        inclination=inclina,
                        heading=Headin,
                        name=f"{tomorrow.month}/{tomorrow.day}",
                    )
                    flights.append(flight)
                    A12 = False
                except:
                    print("error, try again...")
                    err+=1
                    if err == 5:
                        break
        output.clear_output()
        print('1. Launch Date Recommmand')
        print()
        print("Number of flights: ", len(flights))
        comparison = CompareFlights(flights)
        comparison.trajectories_3d(legend=True)
        display(restart_Button)
        
def f2_clicked(b):
    output.clear_output()
    with output:
        print('2.Inclination/Heading comparison')
        print()
        display(Incli)
        display(Headi)
        
def f3_clicked(b):
    output.clear_output()
    with output:
        print('3.Rail Length Comparison')
        print()
        display(Rail_lens)
        display(Rail_Button)
            
def Rail_clicked(b):
    global Rail_lens
    Rail_lens.close()
    Rail_lens = Rail_lens.value
    output.clear_output()
    
    with output:
        print('3.Rail Length Comparison')
        print()
        flights_3 = []
        err = 0
        
        A12 = True

        for i in range(-50,60,10):
            if err == 5:
                break
                
            err = 0
            while A12:
                try:
                    print("Calculating Next Day Weather")
                    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
                    env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
                    env.set_atmospheric_model(type="Forecast", file="GFS")
                    A12 = False

                except:
                    print("error, try again...")
                    err+=1
                    if err == 5:
                        break
            Rail_ = Rail_lens
            Rail_ = Rail_lens + i
            if Rail_ < 0:
                Rail_ = 0.1
            print("Launching Rail {0}".format(Rail_))
            
            flight = Flight(
                environment=env,
                rocket=calisto,
                rail_length=Rail_,
                inclination=inclina,
                heading=Headin,
                name=f"Rail: {Rail_}",
            )
            flights_3.append(flight)
            
        output.clear_output()
        print('3.Rail Length Comparison')
        print()
        
        print("Number of flights: ", len(flights_3))
        comparison_3 = CompareFlights(flights_3)
        
        comparison_3.trajectories_3d(legend=True)
        
        comparison_3.positions()
        display(restart_Button)


Rail_lens = widgets.FloatText(
    value=5.2,
    description='Rail Length',
    disabled=False
)

Rail_Button = widgets.Button(description='Rail Define')
Incli = widgets.Button(description='1.Inclination')
Headi = widgets.Button(description='2.Heading')

def Incli_clicked(b):
    output.clear_output()
    with output:
        print('Inclination comparison')
        print()
        
        flights_2 = []
        err = 0
        
        A12 = True
        global inclina
        inclinat = inclina
        
        for i in range(-10,12,2):
            if err == 5:
                break
            
            err = 0
            while A12:
                try:
                    print("Calculating Next Day Weather...")
                    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
                    env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
                    env.set_atmospheric_model(type="Forecast", file="GFS")
                    A12 = False

                except:
                    print("error, try again...")
                    err+=1
                    if err == 5:
                        break
            print("Launching Inclination {0}°".format(inclinat + i))
            inclina = inclinat + i
            flight = Flight(
                environment=env,
                rocket=calisto,
                rail_length=5.2,
                inclination=inclina,
                heading=Headin,
                name=f"Inclination: {inclina}",
            )
            flights_2.append(flight)
        output.clear_output()
        print('Inclination comparison')
        print()
        print("Number of flights: ", len(flights_2))
        comparison = CompareFlights(flights_2)
        comparison.trajectories_3d(legend=True)
        display(restart_Button)

def Headi_clicked(b):
    output.clear_output()
    with output:
        print('Heading comparison')
        print()
        
        flights_2 = []
        err = 0
        
        A12 = True
        global Headin
        hea = Headin
        
        for i in range(-25,30,5):
            if err == 5:
                break
            
            err = 0
            while A12:
                try:
                    print("Calculating Next Day Weather...")
                    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
                    env.set_date((tomorrow.year, tomorrow.month, tomorrow.day, 12))
                    env.set_atmospheric_model(type="Forecast", file="GFS")
                    A12 = False

                except:
                    print("error, try again...")
                    err+=1
                    if err == 5:
                        break
            print("Launching Heading {0}°".format(hea + i))
            Headin = hea + i
            flight = Flight(
                environment=env,
                rocket=calisto,
                rail_length=5.2,
                inclination=inclina,
                heading=Headin,
                name=f"Heading: {Headin}",
            )
            flights_2.append(flight)
        output.clear_output()
        print('Heading comparison')
        print()
        print("Number of flights: ", len(flights_2))
        comparison = CompareFlights(flights_2)
        comparison.trajectories_3d(legend=True)
        display(restart_Button)


f1.on_click(f1_clicked)
f2.on_click(f2_clicked)
f3.on_click(f3_clicked)
Incli.on_click(Incli_clicked)
Headi.on_click(Headi_clicked)
Rail_Button.on_click(Rail_clicked)

restart_Button = widgets.Button(description='Restart')

def restart_clicked(b):
    output.clear_output()
    print('restart')
    
restart_Button.on_click(restart_clicked)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ToggleButton(value=False, description='Custom Launch Location?', icon='check', tooltip='Using Custom Location'…

Output()

Button(description='Confirm', style=ButtonStyle())